In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Input, AveragePooling2D
from keras import backend as K
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
%matplotlib inline


# from python_speech_features import mfcc
# from python_speech_features import delta
# from python_speech_features import logfbank

from preprocessing import readFileAndAggregateUtterance, getFeature, calculate_XY, conv2D_AvePool
from keras.utils.np_utils import to_categorical
import python_speech_features
import datetime

# Build model
batch_size = 128
num_classes = 4
epochs = 100
kernalSize = 32
categories = ["Anger", "Happiness", "Neutral", "Sadness"]

Using TensorFlow backend.


In [3]:
wavDirBase = "/Users/Chen/Developer/Repository/Clevo-Emotion-Detection-Service/serverless/dev/CNNClassifier2/Preproc/"


x_all, y_all = calculate_XY(wavDirBase, categories, kernalSize, numOfWavsForEachCategory=600)

print(x_all.shape)
print(y_all.shape)

np.savetxt("IEMOCAP_X", x_all)
np.savetxt("IEMOCAP_Y", y_all)



Start processing at 2017-10-11 00:08:40.453519
len(waveArr) 3567
100 files have been processed at 2017-10-11 00:08:51.775339
200 files have been processed at 2017-10-11 00:08:59.947372
300 files have been processed at 2017-10-11 00:09:08.041895
400 files have been processed at 2017-10-11 00:09:16.140394
500 files have been processed at 2017-10-11 00:09:24.873050
600 files have been processed at 2017-10-11 00:09:33.841196
len(waveArr) 657
700 files have been processed at 2017-10-11 00:09:42.833571
800 files have been processed at 2017-10-11 00:09:51.778630
900 files have been processed at 2017-10-11 00:10:00.447008
1000 files have been processed at 2017-10-11 00:10:09.887645
1100 files have been processed at 2017-10-11 00:10:19.122412
1200 files have been processed at 2017-10-11 00:10:27.989771
len(waveArr) 1551
1300 files have been processed at 2017-10-11 00:10:36.836657
1400 files have been processed at 2017-10-11 00:10:45.863557
1500 files have been processed at 2017-10-11 00:10:54.5

In [4]:
# np.savetxt("IEMOCAP_X", x_all)
# np.savetxt("IEMOCAP_Y", y_all)

x_all = np.loadtxt("IEMOCAP_X")
y_all = np.loadtxt("IEMOCAP_Y")

In [14]:
# import numpy
# # x is your dataset
# x = numpy.random.randint(x_all.shape[0], size = x_all.shape[0])
# numpy.random.shuffle(x)
# training, test = x[:int(x_all.shape[0]*0.8)], x[int(x_all.shape[0]*0.8):]
# x_train = x_all[training,:]
# y_train = y_all[training]
# x_test = x_all[test,:]
# y_test = y_all[test]

In [16]:
# # x_train.shape
# x_test.shape

(224, 32)

In [5]:
# print("x_train", x_train[0:3])
# print("y_train", y_train)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=0)

In [6]:
Y_train_cat = to_categorical(Y_train)
Y_test_cat = to_categorical(Y_test)


In [7]:
print(X_train.shape)
print(Y_train_cat.shape)


(1923, 32)
(1923, 4)


In [8]:
# This returns a tensor
lastModel = Sequential()
lastModel.add(Dense(units=num_classes, input_dim=kernalSize))
lastModel.add(Activation('softmax'))
# lastModel.add(Dense(10, activation='softmax'))
lastModel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



In [10]:
lastModel.fit(X_train, Y_train_cat, batch_size=batch_size, epochs=500, verbose=1, validation_split=0.2)

Train on 1538 samples, validate on 385 samples
Epoch 1/500
1538/1538 [==============================] - 0s - loss: 1.3584 - acc: 0.3427 - val_loss: 1.4027 - val_acc: 0.2727
Epoch 2/500
1538/1538 [==============================] - 0s - loss: 1.3581 - acc: 0.3375 - val_loss: 1.4031 - val_acc: 0.2753
Epoch 3/500
1538/1538 [==============================] - 0s - loss: 1.3586 - acc: 0.3388 - val_loss: 1.4036 - val_acc: 0.2909
Epoch 4/500
1538/1538 [==============================] - 0s - loss: 1.3586 - acc: 0.3368 - val_loss: 1.4039 - val_acc: 0.2935
Epoch 5/500
1538/1538 [==============================] - 0s - loss: 1.3585 - acc: 0.3375 - val_loss: 1.4021 - val_acc: 0.2857
Epoch 6/500
1538/1538 [==============================] - 0s - loss: 1.3582 - acc: 0.3446 - val_loss: 1.4032 - val_acc: 0.2779
Epoch 7/500
1538/1538 [==============================] - 0s - loss: 1.3586 - acc: 0.3355 - val_loss: 1.4029 - val_acc: 0.2779
Epoch 8/500
1538/1538 [==============================] - 0s - loss: 1.3

1538/1538 [==============================] - 0s - loss: 1.3581 - acc: 0.3355 - val_loss: 1.4034 - val_acc: 0.2727
Epoch 65/500
1538/1538 [==============================] - 0s - loss: 1.3581 - acc: 0.3375 - val_loss: 1.4026 - val_acc: 0.2701
Epoch 66/500
1538/1538 [==============================] - 0s - loss: 1.3579 - acc: 0.3394 - val_loss: 1.4034 - val_acc: 0.2753
Epoch 67/500
1538/1538 [==============================] - 0s - loss: 1.3582 - acc: 0.3375 - val_loss: 1.4026 - val_acc: 0.2753
Epoch 68/500
1538/1538 [==============================] - 0s - loss: 1.3584 - acc: 0.3322 - val_loss: 1.4021 - val_acc: 0.2883
Epoch 69/500
1538/1538 [==============================] - 0s - loss: 1.3582 - acc: 0.3316 - val_loss: 1.4028 - val_acc: 0.2779
Epoch 70/500
1538/1538 [==============================] - 0s - loss: 1.3581 - acc: 0.3349 - val_loss: 1.4029 - val_acc: 0.2727
Epoch 71/500
1538/1538 [==============================] - 0s - loss: 1.3581 - acc: 0.3407 - val_loss: 1.4038 - val_acc: 0.27

1538/1538 [==============================] - 0s - loss: 1.3585 - acc: 0.3427 - val_loss: 1.4001 - val_acc: 0.2779
Epoch 129/500
1538/1538 [==============================] - 0s - loss: 1.3584 - acc: 0.3401 - val_loss: 1.3998 - val_acc: 0.2779
Epoch 130/500
1538/1538 [==============================] - 0s - loss: 1.3585 - acc: 0.3349 - val_loss: 1.3989 - val_acc: 0.2909
Epoch 131/500
1538/1538 [==============================] - 0s - loss: 1.3584 - acc: 0.3329 - val_loss: 1.3979 - val_acc: 0.2909
Epoch 132/500
1538/1538 [==============================] - 0s - loss: 1.3586 - acc: 0.3322 - val_loss: 1.3979 - val_acc: 0.2857
Epoch 133/500
1538/1538 [==============================] - 0s - loss: 1.3584 - acc: 0.3349 - val_loss: 1.3977 - val_acc: 0.2883
Epoch 134/500
1538/1538 [==============================] - 0s - loss: 1.3584 - acc: 0.3349 - val_loss: 1.3988 - val_acc: 0.2857
Epoch 135/500
1538/1538 [==============================] - 0s - loss: 1.3586 - acc: 0.3349 - val_loss: 1.4002 - val_ac

1538/1538 [==============================] - 0s - loss: 1.3584 - acc: 0.3381 - val_loss: 1.4003 - val_acc: 0.2935
Epoch 192/500
1538/1538 [==============================] - 0s - loss: 1.3585 - acc: 0.3368 - val_loss: 1.3994 - val_acc: 0.2779
Epoch 193/500
1538/1538 [==============================] - 0s - loss: 1.3582 - acc: 0.3446 - val_loss: 1.3988 - val_acc: 0.2909
Epoch 194/500
1538/1538 [==============================] - 0s - loss: 1.3582 - acc: 0.3375 - val_loss: 1.3992 - val_acc: 0.2857
Epoch 195/500
1538/1538 [==============================] - 0s - loss: 1.3584 - acc: 0.3433 - val_loss: 1.3981 - val_acc: 0.2935
Epoch 196/500
1538/1538 [==============================] - 0s - loss: 1.3583 - acc: 0.3446 - val_loss: 1.3964 - val_acc: 0.2883
Epoch 197/500
1538/1538 [==============================] - 0s - loss: 1.3583 - acc: 0.3446 - val_loss: 1.3971 - val_acc: 0.2935
Epoch 198/500
1538/1538 [==============================] - 0s - loss: 1.3588 - acc: 0.3368 - val_loss: 1.3978 - val_ac

1538/1538 [==============================] - 0s - loss: 1.3584 - acc: 0.3355 - val_loss: 1.3972 - val_acc: 0.2857
Epoch 256/500
1538/1538 [==============================] - 0s - loss: 1.3583 - acc: 0.3362 - val_loss: 1.3993 - val_acc: 0.2779
Epoch 257/500
1538/1538 [==============================] - 0s - loss: 1.3583 - acc: 0.3303 - val_loss: 1.3995 - val_acc: 0.2805
Epoch 258/500
1538/1538 [==============================] - 0s - loss: 1.3582 - acc: 0.3336 - val_loss: 1.4000 - val_acc: 0.2701
Epoch 259/500
1538/1538 [==============================] - 0s - loss: 1.3581 - acc: 0.3401 - val_loss: 1.4000 - val_acc: 0.2649
Epoch 260/500
1538/1538 [==============================] - 0s - loss: 1.3584 - acc: 0.3446 - val_loss: 1.4016 - val_acc: 0.2779
Epoch 261/500
1538/1538 [==============================] - 0s - loss: 1.3582 - acc: 0.3453 - val_loss: 1.3986 - val_acc: 0.2727
Epoch 262/500
1538/1538 [==============================] - 0s - loss: 1.3579 - acc: 0.3414 - val_loss: 1.3984 - val_ac

1538/1538 [==============================] - 0s - loss: 1.3583 - acc: 0.3349 - val_loss: 1.4005 - val_acc: 0.2675
Epoch 319/500
1538/1538 [==============================] - 0s - loss: 1.3582 - acc: 0.3316 - val_loss: 1.4013 - val_acc: 0.2779
Epoch 320/500
1538/1538 [==============================] - 0s - loss: 1.3578 - acc: 0.3270 - val_loss: 1.4005 - val_acc: 0.2753
Epoch 321/500
1538/1538 [==============================] - 0s - loss: 1.3580 - acc: 0.3362 - val_loss: 1.4004 - val_acc: 0.2753
Epoch 322/500
1538/1538 [==============================] - 0s - loss: 1.3583 - acc: 0.3420 - val_loss: 1.3998 - val_acc: 0.2805
Epoch 323/500
1538/1538 [==============================] - 0s - loss: 1.3577 - acc: 0.3401 - val_loss: 1.3987 - val_acc: 0.2779
Epoch 324/500
1538/1538 [==============================] - 0s - loss: 1.3579 - acc: 0.3322 - val_loss: 1.3998 - val_acc: 0.2831
Epoch 325/500
1538/1538 [==============================] - 0s - loss: 1.3585 - acc: 0.3388 - val_loss: 1.4008 - val_ac

1538/1538 [==============================] - 0s - loss: 1.3580 - acc: 0.3355 - val_loss: 1.4044 - val_acc: 0.2753
Epoch 383/500
1538/1538 [==============================] - 0s - loss: 1.3580 - acc: 0.3342 - val_loss: 1.4060 - val_acc: 0.2831
Epoch 384/500
1538/1538 [==============================] - 0s - loss: 1.3581 - acc: 0.3414 - val_loss: 1.4056 - val_acc: 0.2831
Epoch 385/500
1538/1538 [==============================] - 0s - loss: 1.3580 - acc: 0.3414 - val_loss: 1.4068 - val_acc: 0.2753
Epoch 386/500
1538/1538 [==============================] - 0s - loss: 1.3579 - acc: 0.3381 - val_loss: 1.4071 - val_acc: 0.2675
Epoch 387/500
1538/1538 [==============================] - 0s - loss: 1.3578 - acc: 0.3394 - val_loss: 1.4071 - val_acc: 0.2701
Epoch 388/500
1538/1538 [==============================] - 0s - loss: 1.3579 - acc: 0.3342 - val_loss: 1.4072 - val_acc: 0.2701
Epoch 389/500
1538/1538 [==============================] - 0s - loss: 1.3582 - acc: 0.3349 - val_loss: 1.4055 - val_ac

1538/1538 [==============================] - 0s - loss: 1.3581 - acc: 0.3388 - val_loss: 1.4011 - val_acc: 0.2701
Epoch 447/500
1538/1538 [==============================] - 0s - loss: 1.3583 - acc: 0.3342 - val_loss: 1.4039 - val_acc: 0.2883
Epoch 448/500
1538/1538 [==============================] - 0s - loss: 1.3583 - acc: 0.3355 - val_loss: 1.4035 - val_acc: 0.2623
Epoch 449/500
1538/1538 [==============================] - 0s - loss: 1.3583 - acc: 0.3362 - val_loss: 1.4035 - val_acc: 0.2675
Epoch 450/500
1538/1538 [==============================] - 0s - loss: 1.3579 - acc: 0.3290 - val_loss: 1.4006 - val_acc: 0.2883
Epoch 451/500
1538/1538 [==============================] - 0s - loss: 1.3582 - acc: 0.3355 - val_loss: 1.4029 - val_acc: 0.2753
Epoch 452/500
1538/1538 [==============================] - 0s - loss: 1.3582 - acc: 0.3414 - val_loss: 1.4041 - val_acc: 0.2805
Epoch 453/500
1538/1538 [==============================] - 0s - loss: 1.3580 - acc: 0.3381 - val_loss: 1.4027 - val_ac

In [11]:
score = lastModel.evaluate(X_test, Y_test_cat, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.41096690861
Test accuracy: 0.266112266112


In [9]:
lastModel.save_weights('emotion_model_weights.h5')

In [44]:
newWavPath = "/Users/Chen/Developer/Repository/Clevo-Emotion-Detection-Service/serverless/dev/CNNClassifier2/Preproc/Sadness/Ses03F_impro02_M017.wav"
result = conv2D_AvePool(newWavPath, kernalSize)
# print(result)

pred_result = lastModel.predict(np.reshape(result, (1,32)), batch_size)[0]
print(pred_result)
print("Prediction result: {}".format(build_index_label(pred_result, categories)))

[ 0.55975288  0.0927767   0.21751454  0.12995584]
Prediction result: Anger


(32,)

In [46]:
def build_index_label(pred, label_list):
    a = max([(v,i) for i,v in enumerate(pred)])
    idx = a[1]
    return label_list[idx]

In [54]:
getFeature("/Users/Chen/Developer/Repository/Clevo-Emotion-Detection-Service/serverless/dev/CNNClassifier2/Preproc/Anger/Ses01F_impro01_M002.wav")

(287, 52)


array([[  5.11733071e+00,   5.98800769e+00,   7.77893292e+00, ...,
         -2.88159901e-02,   8.42821803e-03,   6.96395883e-02],
       [  6.33712011e+00,   7.20940164e+00,   7.77566057e+00, ...,
         -2.62853636e-02,  -1.72670064e-02,  -5.72414303e-02],
       [  6.17165078e+00,   7.18005942e+00,   7.40502920e+00, ...,
         -1.97783151e-03,  -1.31986642e-01,  -2.27144249e-01],
       ..., 
       [  6.36439638e+00,   7.64017045e+00,   7.26427950e+00, ...,
         -5.02181349e-01,  -3.63221517e-01,   7.43935631e-02],
       [  5.51066499e+00,   7.52681444e+00,   7.58245172e+00, ...,
         -5.29187231e-01,  -3.81412090e-01,   4.85745410e-02],
       [  4.75511896e+00,   6.84501919e+00,   7.59563139e+00, ...,
         -3.83579788e-01,  -2.78546633e-01,   8.01068614e-02]])

In [36]:
wavDir = "/Users/Chen/百度云同步盘/Startup/Clevo/数据/IEMOCAP_full_release/EmotionRecognization/wav"
emoTagsDir = "/Users/Chen/百度云同步盘/Startup/Clevo/数据/IEMOCAP_full_release/EmotionRecognization/emotionTags"

for emoFile in os.listdir(emoTagsDir):
    emoFilePath = "{}/{}".format(emoTagsDir, emoFile)
    readFileAndAggregateUtterance(emoFilePath, wavDir, "Preproc")


Ses01F_impro01_F000.wav is put into path: Preproc/Neutral
Ses01F_impro01_F001.wav is put into path: Preproc/Neutral
Ses01F_impro01_F002.wav is put into path: Preproc/Neutral
Ses01F_impro01_F003.wav is put into path: Preproc/Anger
Ses01F_impro01_F004.wav is put into path: Preproc/Anger
Ses01F_impro01_F005.wav is put into path: Preproc/Neutral
Ses01F_impro01_F006.wav is put into path: Preproc/Anger
Ses01F_impro01_F007.wav is put into path: Preproc/Anger
Ses01F_impro01_F008.wav is put into path: Preproc/Anger
Ses01F_impro01_F009.wav is put into path: Preproc/Anger
Ses01F_impro01_F010.wav is put into path: Preproc/Anger
Ses01F_impro01_F011.wav is put into path: Preproc/Anger
Ses01F_impro01_F012.wav is put into path: Preproc/Anger
Ses01F_impro01_F013.wav is put into path: Preproc/Anger
Ses01F_impro01_F015.wav is put into path: Preproc/Anger
Ses01F_impro01_M000.wav is put into path: Preproc/Anger
Ses01F_impro01_M001.wav is put into path: Preproc/Anger
Ses01F_impro01_M002.wav is put into path

Ses01F_impro05_F002.wav is put into path: Preproc/Anger
Ses01F_impro05_F003.wav is put into path: Preproc/Anger
Ses01F_impro05_F004.wav is put into path: Preproc/Anger
Ses01F_impro05_F005.wav is put into path: Preproc/Anger
Ses01F_impro05_F006.wav is put into path: Preproc/Anger
Ses01F_impro05_F007.wav is put into path: Preproc/Anger
Ses01F_impro05_F008.wav is put into path: Preproc/Anger
Ses01F_impro05_F009.wav is put into path: Preproc/Anger
Ses01F_impro05_F010.wav is put into path: Preproc/Anger
Ses01F_impro05_F011.wav is put into path: Preproc/Anger
Ses01F_impro05_F012.wav is put into path: Preproc/Anger
Ses01F_impro05_F013.wav is put into path: Preproc/Anger
Ses01F_impro05_F014.wav is put into path: Preproc/Anger
Ses01F_impro05_F015.wav is put into path: Preproc/Anger
Ses01F_impro05_F016.wav is put into path: Preproc/Anger
Ses01F_impro05_F017.wav is put into path: Preproc/Anger
Ses01F_impro05_F018.wav is put into path: Preproc/Anger
Ses01F_impro05_F019.wav is put into path: Prepro

Ses01F_script01_3_M026.wav is put into path: Preproc/Sadness
Ses01F_script01_3_M027.wav is put into path: Preproc/Sadness
Ses01F_script01_3_M028.wav is put into path: Preproc/Sadness
Ses01F_script01_3_M029.wav is put into path: Preproc/Sadness
Ses01F_script01_3_M030.wav is put into path: Preproc/Sadness
Ses01F_script01_3_M032.wav is put into path: Preproc/Sadness
Ses01F_script01_3_M033.wav is put into path: Preproc/Sadness
Ses01F_script01_3_M036.wav is put into path: Preproc/Sadness
Ses01F_script01_3_M037.wav is put into path: Preproc/Neutral
Ses01F_script02_1_F000.wav is put into path: Preproc/Neutral
Ses01F_script02_1_F001.wav is put into path: Preproc/Anger
Ses01F_script02_1_F004.wav is put into path: Preproc/Neutral
Ses01F_script02_1_F005.wav is put into path: Preproc/Anger
Ses01F_script02_1_F006.wav is put into path: Preproc/Anger
Ses01F_script02_1_F007.wav is put into path: Preproc/Anger
Ses01F_script02_1_F008.wav is put into path: Preproc/Neutral
Ses01F_script02_1_F009.wav is pu

Ses01F_script03_2_M040.wav is put into path: Preproc/Anger
Ses01M_impro01_F000.wav is put into path: Preproc/Anger
Ses01M_impro01_F001.wav is put into path: Preproc/Anger
Ses01M_impro01_F006.wav is put into path: Preproc/Anger
Ses01M_impro01_F007.wav is put into path: Preproc/Anger
Ses01M_impro01_F008.wav is put into path: Preproc/Anger
Ses01M_impro01_F011.wav is put into path: Preproc/Anger
Ses01M_impro01_F012.wav is put into path: Preproc/Anger
Ses01M_impro01_F013.wav is put into path: Preproc/Anger
Ses01M_impro01_F014.wav is put into path: Preproc/Anger
Ses01M_impro01_F015.wav is put into path: Preproc/Anger
Ses01M_impro01_F016.wav is put into path: Preproc/Anger
Ses01M_impro01_F019.wav is put into path: Preproc/Anger
Ses01M_impro01_F020.wav is put into path: Preproc/Anger
Ses01M_impro01_F021.wav is put into path: Preproc/Anger
Ses01M_impro01_F022.wav is put into path: Preproc/Anger
Ses01M_impro01_F024.wav is put into path: Preproc/Neutral
Ses01M_impro01_F029.wav is put into path: P

Ses01M_script01_1_F028.wav is put into path: Preproc/Anger
Ses01M_script01_1_F029.wav is put into path: Preproc/Anger
Ses01M_script01_1_F030.wav is put into path: Preproc/Anger
Ses01M_script01_1_F031.wav is put into path: Preproc/Anger
Ses01M_script01_1_F031.wav is put into path: Preproc/Sadness
Ses01M_script01_1_F032.wav is put into path: Preproc/Anger
Ses01M_script01_1_F034.wav is put into path: Preproc/Anger
Ses01M_script01_1_F035.wav is put into path: Preproc/Anger
Ses01M_script01_1_F036.wav is put into path: Preproc/Anger
Ses01M_script01_1_F037.wav is put into path: Preproc/Anger
Ses01M_script01_1_F038.wav is put into path: Preproc/Anger
Ses01M_script01_1_F039.wav is put into path: Preproc/Anger
Ses01M_script01_1_F040.wav is put into path: Preproc/Anger
Ses01M_script01_1_F041.wav is put into path: Preproc/Anger
Ses01M_script01_1_F042.wav is put into path: Preproc/Anger
Ses01M_script01_1_F043.wav is put into path: Preproc/Anger
Ses01M_script01_1_M000.wav is put into path: Preproc/N

...

Ses05F_impro08_F012.wav is put into path: Preproc/Neutral
Ses05F_impro08_F014.wav is put into path: Preproc/Neutral
Ses05F_impro08_F015.wav is put into path: Preproc/Neutral
Ses05F_impro08_F016.wav is put into path: Preproc/Neutral
Ses05F_impro08_F017.wav is put into path: Preproc/Neutral
Ses05F_impro08_F018.wav is put into path: Preproc/Neutral
Ses05F_impro08_F019.wav is put into path: Preproc/Neutral
Ses05F_impro08_F020.wav is put into path: Preproc/Neutral
Ses05F_impro08_F021.wav is put into path: Preproc/Neutral
Ses05F_impro08_F022.wav is put into path: Preproc/Neutral
Ses05F_impro08_F023.wav is put into path: Preproc/Neutral
Ses05F_impro08_F024.wav is put into path: Preproc/Neutral
Ses05F_impro08_F025.wav is put into path: Preproc/Neutral
Ses05F_impro08_F026.wav is put into path: Preproc/Neutral
Ses05F_impro08_F027.wav is put into path: Preproc/Neutral
Ses05F_impro08_F028.wav is put into path: Preproc/Neutral
Ses05F_impro08_F029.wav is put into path: Preproc/Neutral
Ses05F_impro08

Ses05F_script03_1_M012.wav is put into path: Preproc/Happiness
Ses05F_script03_1_M023.wav is put into path: Preproc/Happiness
Ses05F_script03_1_M030.wav is put into path: Preproc/Neutral
Ses05F_script03_2_F000.wav is put into path: Preproc/Neutral
Ses05F_script03_2_F001.wav is put into path: Preproc/Neutral
Ses05F_script03_2_F003.wav is put into path: Preproc/Neutral
Ses05F_script03_2_F004.wav is put into path: Preproc/Neutral
Ses05F_script03_2_F005.wav is put into path: Preproc/Neutral
Ses05F_script03_2_F006.wav is put into path: Preproc/Neutral
Ses05F_script03_2_F007.wav is put into path: Preproc/Anger
Ses05F_script03_2_F008.wav is put into path: Preproc/Anger
Ses05F_script03_2_F010.wav is put into path: Preproc/Neutral
Ses05F_script03_2_F011.wav is put into path: Preproc/Neutral
Ses05F_script03_2_F012.wav is put into path: Preproc/Neutral
Ses05F_script03_2_F013.wav is put into path: Preproc/Anger
Ses05F_script03_2_F014.wav is put into path: Preproc/Neutral
Ses05F_script03_2_F015.wav

Ses05M_impro04_M032.wav is put into path: Preproc/Anger
Ses05M_impro04_M033.wav is put into path: Preproc/Anger
Ses05M_impro04_M034.wav is put into path: Preproc/Anger
Ses05M_impro04_M035.wav is put into path: Preproc/Anger
Ses05M_impro04_M036.wav is put into path: Preproc/Anger
Ses05M_impro04_M037.wav is put into path: Preproc/Anger
Ses05M_impro04_M038.wav is put into path: Preproc/Anger
Ses05M_impro04_M039.wav is put into path: Preproc/Anger
Ses05M_impro04_M040.wav is put into path: Preproc/Anger
Ses05M_impro04_M041.wav is put into path: Preproc/Anger
Ses05M_impro04_M041.wav is put into path: Preproc/Sadness
Ses05M_impro05_F000.wav is put into path: Preproc/Neutral
Ses05M_impro05_F001.wav is put into path: Preproc/Neutral
Ses05M_impro05_F002.wav is put into path: Preproc/Neutral
Ses05M_impro05_F003.wav is put into path: Preproc/Neutral
Ses05M_impro05_F004.wav is put into path: Preproc/Neutral
Ses05M_impro05_F005.wav is put into path: Preproc/Neutral
Ses05M_impro05_F006.wav is put int

Ses05M_script01_1b_M002.wav is put into path: Preproc/Sadness
Ses05M_script01_1b_M003.wav is put into path: Preproc/Sadness
Ses05M_script01_1b_M004.wav is put into path: Preproc/Sadness
Ses05M_script01_1b_M005.wav is put into path: Preproc/Sadness
Ses05M_script01_1b_M008.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M009.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M010.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M012.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M016.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M017.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M018.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M019.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M020.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M021.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M022.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M025.wav is put into path: Preproc/Anger
Ses05M_script01_1b_M026.wav is p

Ses05M_script03_2_M012.wav is put into path: Preproc/Anger
Ses05M_script03_2_M013.wav is put into path: Preproc/Anger
Ses05M_script03_2_M014.wav is put into path: Preproc/Anger
Ses05M_script03_2_M015.wav is put into path: Preproc/Anger
Ses05M_script03_2_M016.wav is put into path: Preproc/Anger
Ses05M_script03_2_M017.wav is put into path: Preproc/Anger
Ses05M_script03_2_M018.wav is put into path: Preproc/Anger
Ses05M_script03_2_M019.wav is put into path: Preproc/Anger
Ses05M_script03_2_M020.wav is put into path: Preproc/Anger
Ses05M_script03_2_M021.wav is put into path: Preproc/Anger
Ses05M_script03_2_M022.wav is put into path: Preproc/Anger
Ses05M_script03_2_M023.wav is put into path: Preproc/Anger
Ses05M_script03_2_M024.wav is put into path: Preproc/Neutral
Ses05M_script03_2_M025.wav is put into path: Preproc/Anger
Ses05M_script03_2_M026.wav is put into path: Preproc/Anger
Ses05M_script03_2_M027.wav is put into path: Preproc/Anger
Ses05M_script03_2_M028.wav is put into path: Preproc/A

In [34]:
(rate,sig) = wav.read("20170623170919_642_15842777828_601.wav")

print("rate", rate)

#  - Mel Frequency Cepstral Coefficients
mfcc_feat = python_speech_features.mfcc(sig,rate)
d_mfcc_feat = python_speech_features.delta(mfcc_feat, 2)
# - Filterbank Energies
fbank_feat = python_speech_features.fbank(sig,rate)[0]
# - Log Filterbank Energies
logfbank_feat = python_speech_features.logfbank(sig,rate)

# - Spectral Subband Centroids
ssc_feat = python_speech_features.ssc(sig,rate)

# mfcc_feat = mfcc(sig,rate)
# d_mfcc_feat = delta(mfcc_feat, 2)

# fbank_feat = logfbank(sig,rate)



rate 8000


In [36]:
print("mfcc_feat shape: ", mfcc_feat.shape)
print("fbank_feat shape: ", fbank_feat.shape)
print("logfbank_feat shape: ", logfbank_feat.shape)
print("ssc_feat shape: ", ssc_feat.shape)

mfcc_feat shape:  (17171, 13)
fbank_feat shape:  (17171, 26)
logfbank_feat shape:  (17171, 26)
ssc_feat shape:  (17171, 26)


In [38]:
sample_start = 1231

print("mfcc_feat")
print(mfcc_feat[sample_start:sample_start+3, :])

print("fbank_feat")
print(fbank_feat[sample_start:sample_start+3, :])

print("logfbank_feat")
print(logfbank_feat[sample_start:sample_start+3, :])

print("ssc_feat")
print(ssc_feat[sample_start:sample_start+3, :])




mfcc_feat
[[ 16.13379134   4.61542291 -41.76135789  -5.2807745   10.73237904
    3.61484241 -18.79295092  -1.82976924  -1.99055823 -31.75436637
    8.71127836  -1.95209103  -7.79984302]
 [ 16.11312882   6.97780363 -46.12920588   0.576429     8.11739545
   13.63711798 -10.70017864  -4.37646628  -6.66814631 -25.61086489
    4.36502522   0.18921123  -6.92343554]
 [ 15.86549092  -5.54006824 -45.48896869  -9.83534552 -20.15050215
    0.43181621 -34.00344988 -20.773685   -19.43556872 -33.14515784
   -0.83002269   4.36631977   4.50879581]]
fbank_feat
[[  1.92401208e+03   2.54199852e+03   1.38746486e+04   5.50024282e+04
    5.24447181e+03   1.67469111e+04   8.55415434e+04   2.09275494e+04
    1.15860422e+05   1.34061197e+06   7.52085282e+05   1.41970055e+05
    4.12949652e+06   1.84099327e+06   4.18205933e+05   9.82823130e+05
    5.53800686e+04   2.36027224e+04   4.71348723e+03   3.69242672e+04
    9.68200494e+04   4.20049380e+03   2.82439337e+03   2.42451414e+03
    2.28003317e+03   2.4003720

In [39]:
mfcc_feat.shape

(17171, 13)

In [40]:
print(d_mfcc_feat[1:3,:])

[[  0.00000000e+00   0.00000000e+00   1.26217745e-30   0.00000000e+00
   -1.00974196e-29   0.00000000e+00  -5.04870979e-30   0.00000000e+00
   -1.00974196e-29   0.00000000e+00   1.00974196e-29   0.00000000e+00
    1.00974196e-29]
 [  0.00000000e+00   0.00000000e+00   1.26217745e-30   0.00000000e+00
   -1.00974196e-29   0.00000000e+00  -5.04870979e-30   0.00000000e+00
   -1.00974196e-29   0.00000000e+00   1.00974196e-29   0.00000000e+00
    1.00974196e-29]]


In [16]:
print(fbank_feat[1:3,:])

[[-36.04365339 -36.04365339 -36.04365339 -36.04365339 -36.04365339
  -36.04365339 -36.04365339 -36.04365339 -36.04365339 -36.04365339
  -36.04365339 -36.04365339 -36.04365339 -36.04365339 -36.04365339
  -36.04365339 -36.04365339 -36.04365339 -36.04365339 -36.04365339
  -36.04365339 -36.04365339 -36.04365339 -36.04365339 -36.04365339
  -36.04365339]
 [-36.04365339 -36.04365339 -36.04365339 -36.04365339 -36.04365339
  -36.04365339 -36.04365339 -36.04365339 -36.04365339 -36.04365339
  -36.04365339 -36.04365339 -36.04365339 -36.04365339 -36.04365339
  -36.04365339 -36.04365339 -36.04365339 -36.04365339 -36.04365339
  -36.04365339 -36.04365339 -36.04365339 -36.04365339 -36.04365339
  -36.04365339]]


In [9]:
d_mfcc_feat.head(3)

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [3]:

# get the data
x_train, y_train, paths_train, x_test, y_test, paths_test, class_names, sr = build_datasets(preproc=True)

# # input image dimensions
img_rows, img_cols = 96, 157




class_names =  ['Love, Happiness', 'Sadness, Sorrow']
total files =  46
os.path.exists(audio_path) True
   get_sample_dimensions: melgram.shape =  (1, 1, 96, 157)

 Loading class: Love, Happiness ( 1 of  2 classes), file 1 of 24: Preproc/Love, Happiness/334490_209_091532__3.wav.npy


 Loading class: Sadness, Sorrow ( 2 of  2 classes), file 1 of 22: Preproc/Sadness, Sorrow/334491_208_091642__3.wav.npy

Shuffling order of data...


In [90]:

# plt.figure(figsize=(10,10))
# start = 9;
# plt.subplot(221)
# plt.imshow(x_train[start+0,:,:,0])
# plt.subplot(222)
# plt.imshow(x_train[start+1,:,:,0])
# plt.subplot(223)
# plt.imshow(x_train[start+2,:,:,0])
# plt.subplot(224)
# plt.imshow(x_train[start+3,:,:,0])

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
#           validation_split = 0.2
          validation_data=(x_test, y_test)
         )
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 36 samples, validate on 10 samples
Epoch 1/12
36/36 [==============================] - 3s - loss: 0.6919 - acc: 0.5278 - val_loss: 0.6914 - val_acc: 0.5000
Epoch 2/12
36/36 [==============================] - 2s - loss: 0.6722 - acc: 0.6111 - val_loss: 0.6838 - val_acc: 0.6000
Epoch 3/12
36/36 [==============================] - 2s - loss: 0.6476 - acc: 0.6111 - val_loss: 0.7029 - val_acc: 0.5000
Epoch 4/12
36/36 [==============================] - 2s - loss: 0.6763 - acc: 0.5833 - val_loss: 0.6676 - val_acc: 0.5000
Epoch 5/12
36/36 [==============================] - 2s - loss: 0.6365 - acc: 0.7500 - val_loss: 0.7203 - val_acc: 0.5000
Epoch 6/12
36/36 [==============================] - 2s - loss: 0.6710 - acc: 0.5278 - val_loss: 0.6663 - val_acc: 0.6000
Epoch 7/12
36/36 [==============================] - 2s - loss: 0.6095 - acc: 0.6944 - val_loss: 0.6642 - val_acc: 0.5000
Epoch 8/12
36/36 [==============================] - 2s - loss: 0.5568 - acc: 0.6944 - val_loss: 0.6368 - val_